In [8]:
# vamos importar o selenium porque com ele conseguimos mexer com os navegadores
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
import time
import zipfile
import pandas as pd
import win32com.client as win32

# passo 1 - abrindo o navegador e colocando o link que vai ser aberto
driver = webdriver.Chrome()
driver.get('https://www.kaggle.com/sakshigoyal7/credit-card-customers')

# clicando em um elemento (primeiro elemento da lista) pelo css selector
driver.find_elements_by_css_selector('a.sc-fzoYHE')[0].click()

# vai esperar 1 segundo
time.sleep(1)

# clicando em um elemento (segundo elemento da lista) pela classe
driver.find_elements_by_class_name('sc-jWtpDG')[1].click()
time.sleep(1)

# escrevendo valores com o .send_keys
driver.find_elements_by_css_selector('input.mdc-text-field__input')[0].send_keys('joaoprlira@gmail.com')
driver.find_elements_by_css_selector('input.mdc-text-field__input')[1].send_keys('zxc123456')
driver.find_element_by_css_selector('form.sc-kIGZdZ').submit()
time.sleep(2)

# clicando para baixar o arquivo
driver.find_elements_by_css_selector('a.sc-fzoYHE')[0].click()
time.sleep(5)


# esse processo mostra como tirar um arquivo do zip
with zipfile.ZipFile(r'C:\Users\joaop\Downloads\archive.zip', 'r') as zip_ref:
    zip_ref.extractall(r"C:\Users\joaop\Downloads")
    
    
# lendo o arquivo csv
clientes_df = pd.read_csv(r'C:\Users\joaop\Downloads\BankChurners.csv')
# display(clients_df)

# 1 - agrupou os valores da tabela que contou quantos usuários existem e quantos não existem
resumo_status = clientes_df.groupby('Attrition_Flag')['Attrition_Flag'].count()
#print(resumo_status.to_string())

#2 - pegando linhas e colunas
distribuicao_cartao = clientes_df.loc[clientes_df['Attrition_Flag']=='Existing Customer', ['Attrition_Flag', 'Card_Category']]
#display(distribuicao_cartao)

# vai agrupar valores os valores dos cartões e contar tudo
distribuicao_cartao = distribuicao_cartao.groupby('Card_Category')['Card_Category'].count()

# mudou o título da coluna para 'Card_Category - Existing Customer'
distribuicao_cartao.index.names = ["Card_Category - Existing Customer"]
#print(distribuicao_cartao.to_string())

#3 - calculando a média 
tempo_medio = clientes_df['Months_on_book'].mean()
limite_medio = clientes_df['Credit_Limit'].mean()
limite_medio_exclientes = clientes_df.loc[clientes_df['Attrition_Flag']=='Attrited Customer', 'Credit_Limit'].mean()
#print(limite_medio)
#print(limite_medio_exclientes)



# enviar um e-mail para o chefe
outlook = win32.Dispatch('outlook.application')

texto = f'''
Olá Lira, tudo bem?

Conforme solicitado, levantamos os principais indicadores dos nossos clientes para ver o impacto dos Attrited Customers.

Temos atualmente a seguinte divisão da base de clientes:

{resumo_status.to_string()}

Além disso, dos clientes ativos (Existing Customers) a divisão por categoria fica assim:

{distribuicao_cartao.to_string()}

Por fim, o tempo médio de permanência dos clientes é de {tempo_medio:.1f} meses. Agora, quando comparamos o limite médio entre clientes ativos e não ativos, não percebemos muita diferenciação, sendo {limite_medio:.1f} para ativos e {limite_medio_exclientes:.1f} para inativos

Segue em anexo planilha completa para mais detalhes.

Att.,
João
'''

mail = outlook.CreateItem(0)
mail.To = 'joaoprlira@gmail.com'
mail.Subject = 'Relatório de Clientes - Análise de Attrited Customers'
mail.Body = texto

attachment  = r'C:\Users\joaop\Downloads\BankChurners.csv'
mail.Attachments.Add(attachment)

mail.Send()

print('Fim')

Fim
